# Data Prep

Code authored by: Shaw Talebi 

[Blog link](https://medium.com/towards-data-science/multimodal-rag-process-any-file-type-with-ai-e6921342c903) 
| [Video link](https://youtu.be/Y7pNmocrmi8)

### imports

In [1]:
from bs4 import BeautifulSoup
import os
from functions import *

from PIL import Image
from transformers import CLIPProcessor, CLIPModel

from torch import cat, save

### Extract text and images

In [2]:
# Extract text and images

root_dir = "raw"

# raw 디렉터리 안에서 "파일"이면서 .html/.htm 인 것만 선택
filename_list = []
for name in os.listdir(root_dir):
    path = os.path.join(root_dir, name)
    if os.path.isfile(path) and path.lower().endswith((".html", ".htm")):
        filename_list.append(path)

print(filename_list)

['raw/2024-11-19_Multimodal-Models---LLMs-that-can-see-and-hear-5c6737c981d3.html', 'raw/2024-11-29_Multimodal-Embeddings--An-Introduction-5dc36975966f.html']


In [3]:
# Get all HTML files from raw directory
#filename_list = ["raw/"+f for f in os.listdir('raw')]

text_content_list = []
image_content_list = []
for filename in filename_list:

    with open(filename, 'r', encoding='utf-8') as file:
        html_content = file.read()

    text_content_list.extend(parse_html_content(html_content))
    image_content_list.extend(parse_html_images(html_content))

In [4]:
print(len(text_content_list))
print(len(image_content_list))

86
17


### You have 17 entries in image_content_list, but if you look closely:

13 https://cdn-images-1.medium.com/max/800/1*4wnqr5p_7N3QD5EkXIQeew.png

15 https://cdn-images-1.medium.com/max/800/1*4wnqr5p_7N3QD5EkXIQeew.png  (duplicate)

12 https://cdn-images-1.medium.com/max/800/1*Nzo536sqahqm1Q24Ms2vmA.png

16 https://cdn-images-1.medium.com/max/800/1*Nzo536sqahqm1Q24Ms2vmA.png   (duplicate)



In [5]:
for i, content in enumerate(image_content_list):
    print(i, content['image_path'], "|", content['caption'])

0 https://cdn-images-1.medium.com/max/800/0*YE-Q-OuWnrgrUrQw | Photo by Sincerely Media on Unsplash
1 https://cdn-images-1.medium.com/max/800/1*yvfu8VAp1UgCw4SVvUe77Q.png | Example mutlimodal models. Image by author.
2 https://cdn-images-1.medium.com/max/800/1*Nwc-ZhRFKH17LWWmsNhbdA.png | An example of information degradation during message passing. Image by author.
3 https://cdn-images-1.medium.com/max/800/1*pyqGh5Cbrk_EMlPYtrfrQw.png | A simple strategy for integrating images into an LLM via an image encoding adapter. Image by author.
4 https://cdn-images-1.medium.com/max/800/1*lvX8Mut8SQ1vDhsaewLQ_g.jpeg | Image of me from networking event at Richardson IQ. Image by author.
5 https://cdn-images-1.medium.com/max/800/1*IqUoZEX2CYOsX6oFIVeuIw.jpeg | Building with AI meme. Image by author.
6 https://cdn-images-1.medium.com/max/800/1*PRSGngwjIVW01cLHK41lNg.jpeg | Screenshot of 5 AI project ideas. Image by author.
7 https://cdn-images-1.medium.com/max/800/1*a6BF-kEeo8rd7OW2a3JYGA.png | Im

## On an HPC system, Compute / login nodes often have no outbound internet or very restricted HTTP(S).

### Step 1: Collect local images sorted numerically

In [6]:
import os

image_dir = "images"

local_files = []
for fname in os.listdir(image_dir):
    name, ext = os.path.splitext(fname)
    if ext.lower() in (".png", ".jpg", ".jpeg"):
        try:
            idx = int(name)  # "1" -> 1
        except ValueError:
            continue  # skip non-numeric names
        
        local_files.append((idx, os.path.join(image_dir, fname)))

# sort by numeric index: 1,2,...,15
local_files.sort(key=lambda x: x[0])
local_paths = [path for _, path in local_files]

print("Local image paths (numeric sorted):")
for p in local_paths:
    print(p)


Local image paths (numeric sorted):
images/1.jpg
images/2.png
images/3.png
images/4.png
images/5.png
images/6.png
images/7.jpeg
images/8.png
images/9.jpeg
images/10.png
images/11.png
images/12.jpeg
images/13.png
images/14.png
images/15.png


### Step 2: Build the list of unique image URLs (in order)

In [7]:
unique_urls = []
for item in image_content_list:
    url = item["image_path"]
    if url not in unique_urls:
        unique_urls.append(url)

print("Unique URLs:", len(unique_urls))
for i, url in enumerate(unique_urls):
    print(i, url)


Unique URLs: 15
0 https://cdn-images-1.medium.com/max/800/0*YE-Q-OuWnrgrUrQw
1 https://cdn-images-1.medium.com/max/800/1*yvfu8VAp1UgCw4SVvUe77Q.png
2 https://cdn-images-1.medium.com/max/800/1*Nwc-ZhRFKH17LWWmsNhbdA.png
3 https://cdn-images-1.medium.com/max/800/1*pyqGh5Cbrk_EMlPYtrfrQw.png
4 https://cdn-images-1.medium.com/max/800/1*lvX8Mut8SQ1vDhsaewLQ_g.jpeg
5 https://cdn-images-1.medium.com/max/800/1*IqUoZEX2CYOsX6oFIVeuIw.jpeg
6 https://cdn-images-1.medium.com/max/800/1*PRSGngwjIVW01cLHK41lNg.jpeg
7 https://cdn-images-1.medium.com/max/800/1*a6BF-kEeo8rd7OW2a3JYGA.png
8 https://cdn-images-1.medium.com/max/800/1*jpmC6Kx7DxVeikEr15vooA.png
9 https://cdn-images-1.medium.com/max/800/1*5d3HBNjNIXLy0oMIvJjxWw.png
10 https://cdn-images-1.medium.com/max/800/1*AGHBVjzwjXapJSe4aUPrjg.png
11 https://cdn-images-1.medium.com/max/800/1*2X1aT8fzFsgbqn23zXmmAA.png
12 https://cdn-images-1.medium.com/max/800/1*Nzo536sqahqm1Q24Ms2vmA.png
13 https://cdn-images-1.medium.com/max/800/1*4wnqr5p_7N3QD5EkXIQe

### Step 3: Create a URL → local file mapping

In [8]:
assert len(unique_urls) == len(local_paths), "Mismatch between unique URLs and local images"

url_to_path = {url: local_paths[i] for i, url in enumerate(unique_urls)}

#### If you want to be extra safe, you can also store the original URL:

In [9]:
for item in image_content_list:
    url = item["image_path"]
    item["source_url"] = url  # keep original URL
    if url in url_to_path:
        item["image_path"] = url_to_path[url]
    else:
        print("WARNING: no local file mapped for", url)


### Step 4: Build image_list safely

In [10]:
from PIL import Image

image_list = []
for content in image_content_list:
    try:
        img = Image.open(content["image_path"]).convert("RGB")
        image_list.append(img)
    except FileNotFoundError:
        print("Missing local file:", content["image_path"])


### Build test_list

In [11]:
text_list = []
for content in text_content_list:
    # concatenate title and section header
    section = content['section'] + ": "
    # append text from paragraph to fill CLIP's 256 sequence limit
    text = section + content['text'][:256-len(section)]
    
    text_list.append(text)

#image_list = []
#for content in image_content_list:
#    image_list.append(Image.open(content['image_path']))

In [12]:
print(len(text_list))
print(len(image_list))

86
17


### Compute embeddings using CLIP

In [13]:
# import model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")

# import processor (handles text tokenization and image preprocessing)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16") 

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [14]:
# pre-process text and images
inputs = processor(text=text_list, images=image_list, return_tensors="pt", padding=True)

In [15]:
# compute embeddings with CLIP
outputs = model(**inputs)

In [16]:
# store embeddings in single torch tensor
text_embeddings = outputs.text_embeds
image_embeddings = outputs.image_embeds

In [17]:
print(text_embeddings.shape)
print(image_embeddings.shape)

torch.Size([86, 512])
torch.Size([17, 512])


### Save Data

In [18]:
# save content list as JSON
save_to_json(text_content_list, output_file='data/text_content.json')
save_to_json(image_content_list, output_file='data/image_content.json')

In [19]:
# save embeddings to file
save(text_embeddings, 'data/text_embeddings.pt')
save(image_embeddings, 'data/image_embeddings.pt')